In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

Carga de datos.

In [ ]:
data = pd.read_csv('../data/processed/incidents_firearm_laws.csv')

Seleccionamos las características a incluir en el PCA. Por querer hacer un análisis más general, no utilizaremos ni el estado ni el año. Tampoco incluiremos el número de incidentes, por ser esta nuestra variable objetivo.

In [ ]:
features = data.iloc[:, 3:]

Estandarizamos los datos.

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(features)

Aplicamos PCA.

In [ ]:
pca = PCA()
pca_result = pca.fit_transform(scaled_data)

Visualizamos la varianza explicada acumulada para decidir cuántos componentes escogemos.

In [ ]:
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_.cumsum(), marker='o')
plt.plot([1, len(pca.explained_variance_ratio_)], [0.95, 0.95], linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.show()

Basándonos en el gráfico, deberíamos escoger más de 9 componentes. Sin embargo, escogemos 3 componentes, de manera que expliquemos la mayor varianza posible a la vez que podamos visualizarlo.

In [ ]:
n_components = 3
pca = PCA(n_components=n_components)
pca_result = pca.fit_transform(scaled_data)

Visualizamos tanto el método del codo como los coeficientes de Silhouette para escoger el número de clusters.

In [ ]:
# Use elbow method and silhouette score to determine optimal number of clusters
from sklearn.metrics import silhouette_score

wcss = []
silhouette_scores = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(pca_result)
    wcss.append(kmeans.inertia_)
    if i > 1:
        silhouette_scores.append(silhouette_score(pca_result, kmeans.labels_))

# Plot elbow method and silhouette score
fig, ax1 = plt.subplots()
ax1.plot(range(1, 11), wcss)
ax1.set_xlabel('Number of Clusters')
ax1.set_ylabel('WCSS')
ax1.set_title('Elbow Method')

ax2 = ax1.twinx()
ax2.plot(range(2, 11), silhouette_scores, color='r')
ax2.set_ylabel('Silhouette Score')

Escogemos 3 clusters.

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(pca_result)

Añadimos el cluster correspondiente a cada registro.

In [ ]:
data['cluster'] = clusters
data

Mostramos la media de cada característica en cada cluster.

In [ ]:
# Group by cluster and calculate mean values for each feature
relevant_data = data.drop(['state', 'year'], axis=1)
cluster_data = relevant_data.groupby('cluster').mean()

# Order clusters by n_incidents
cluster_data = cluster_data.sort_values('n_incidents')
cluster_data

Para un análisis más profundo, mostraremos qué características varían más de un cluster a otro.

In [ ]:
# Get which features vary the most between clusters (without n_incidents)
cluster_data.drop('n_incidents', axis=1).std().sort_values(ascending=False)

Por último, visualizamos los clusters.

In [ ]:
# Plot 3D PCA
%matplotlib widget
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(pca_result[:, 0], pca_result[:, 1], pca_result[:, 2], c=data['cluster'], s=50)
ax.set_title('K-Means Clustering')
ax.set_xlabel('PCA 1')
ax.set_ylabel('PCA 2')
ax.set_zlabel('PCA 3')
plt.colorbar(scatter)
plt.show()